In [1]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import glob
from tqdm.notebook import tqdm

In [2]:
data_path = 'data2'

In [3]:
ssubm_df = pd.read_csv(os.path.join(data_path, 'input', 'sample_submission.csv'))
ssubm_df = ssubm_df['site_path_timestamp'].apply(lambda x: pd.Series(x.split('_')))
ssubm_df.columns = ['site', 'path', 'timestamp']

In [5]:
site_train_paths = glob.glob(os.path.join(data_path, 'output', 'features', 'train', '*'))

feauter_dfs = list()
for site_train_path in tqdm(site_train_paths):
    feauter_dfs.append(pd.read_csv(site_train_path))

  0%|          | 0/24 [00:00<?, ?it/s]

In [6]:
def create_train_df(feauter_df):
    df = feauter_df.copy()
    Y_np = df['floor'].to_numpy()

    df.drop(['x', 'y', 'floor', 'path', 'sys_ts'], axis=1, inplace=True)
    X_np = df.to_numpy()
    
    return X_np, Y_np

In [7]:
models = dict()

for i in tqdm(range(len(site_train_paths))):
    site = site_train_paths[i].split('/')[-1][:-4]
    
    X_np, Y_np = create_train_df(feauter_dfs[i])
    
    model = RandomForestClassifier(n_estimators=20)
    model.fit(X_np, Y_np)
    
    models[site] = model

  0%|          | 0/24 [00:00<?, ?it/s]

In [8]:
site_test_paths = glob.glob(os.path.join(data_path, 'output', 'features', 'test', '*'))

test_feauter_dfs = dict()
for site_test_path in tqdm(site_test_paths):
    site = site_test_path.split('/')[-1][:-4]
    test_feauter_dfs[site] = pd.read_csv(site_test_path)

  0%|          | 0/24 [00:00<?, ?it/s]

In [9]:
for index, row in tqdm(ssubm_df.iterrows(), total=ssubm_df.shape[0]):
    model = models[row.site]
    
    test_site_feauter_df = test_feauter_dfs[row.site]
    test_path_feauter_df = test_site_feauter_df[test_site_feauter_df.path == row.path]
    
    dists = list()
    for i, row_feat in test_path_feauter_df.iterrows():
        dist = abs(int(row_feat.sys_ts) - int(row.timestamp))
        dists.append(dist)
    nearest = test_path_feauter_df.iloc[np.argmin(dists)]
    
    X_np = np.array([nearest[:-2].to_numpy()])
    Y_pred = model.predict(X_np)[0]
    
    ssubm_df.loc[index, ['floor']] = Y_pred
    
    ssubm_df.loc[index, ['x']] = 75
    ssubm_df.loc[index, ['y']] = 75
    
    #break
    

  0%|          | 0/10133 [00:00<?, ?it/s]

In [10]:
ssubm_df['site_path_timestamp'] = ssubm_df['site'] + '_' + ssubm_df['path'] + '_' + ssubm_df['timestamp'] 
ssubm_df.drop(['site', 'path', 'timestamp'], axis=1, inplace=True)

cols = ssubm_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
ssubm_df = ssubm_df[cols]

In [12]:
ssubm_df

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0.0,75.0,75.0
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0.0,75.0,75.0
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0.0,75.0,75.0
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0.0,75.0,75.0
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0.0,75.0,75.0
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5.0,75.0,75.0
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5.0,75.0,75.0
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5.0,75.0,75.0
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5.0,75.0,75.0


In [ ]:
ssubm_df.to_csv(os.path.join(data_path, 'output', 'submission_1.csv'), index=False)